In [1]:
#Imports 
import twint
import json
from bert_serving.client import BertClient
import nest_asyncio
import numpy as np
import re
import json
import sys
import pdb
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import pickle
import nltk
import spacy

In [12]:
#Main For calling all other functions
#Make sure server is running before you start this
nlp = spacy.load('en_core_web_sm')
nest_asyncio.apply()
#Step 1 - Pulling Tweets
tags = getTags()
tags = processTags(tags)
scrapeTweets(tags)
data = processData()
#Step 2 -Collect Tweet Embeddings and Step 3- Pass Embeddings through svm_model.p to get Labels
data = cleanData(data)
# getEncodings(newTweets) Called from cleanData function!
classifyTweets("svm_model.p","new_text.npy","encoded_file.npy","classifiedTweets.txt",data)
#Step 4 - Keep Only Relevant Tweets and Pass through NLTK
data = removeIrrelevant(data)
data = applyNLTK(data)
data = applySpacy(data)
#Step 5 -Send to GraphDB
print("Done!")

1250892818819747840 2020-04-16 14:04:23 Pacific Daylight Time <gmsectec_us> Router vendor Linksys has locked user accounts on its Smart WiFi cloud service and is asking users to reset passwords after #hackers have been observed hijacking accounts and changing router settings to redirect users to #malware sites. #GMsectec https://zd.net/2RMCNDk 
1250892639018160128 2020-04-16 14:03:40 Pacific Daylight Time <IT_securitynews> Shipping giant MSC discloses a malware-based attack  https://www.itsecuritynews.info/shipping-giant-msc-discloses-a-malware-based-attack/ …
1250892267394605057 2020-04-16 14:02:11 Pacific Daylight Time <AyeThatBlasian> They say that VPNs, Antivirus, and Firewalls work hand and hand and u need all of them.. Firewalls protect your device's information (your files) but VPNs protect your personal information that u send via internet, and Antivirus kills viruses on your computer and malware
1250892229721427971 2020-04-16 14:02:02 Pacific Daylight Time <Askcyber01> RAT Mal

CRITICAL:root:twint.output:checkData:copyrightedTweet
CRITICAL:root:twint.output:checkData:copyrightedTweet
CRITICAL:root:twint.output:checkData:copyrightedTweet
CRITICAL:root:twint.output:checkData:copyrightedTweet


734824011713138688 2016-05-23 12:11:11 Pacific Daylight Time <lookatyoubuddy> I get high off plain old living
734740783606161408 2016-05-23 06:40:28 Pacific Daylight Time <Rob0tCunts> Ahhh right. He's decent. If you like his stuff, try JRE by @joerogan
734740364725215232 2016-05-23 06:38:48 Pacific Daylight Time <Rob0tCunts> pretty sure he said something very similar - yeah me too
734739842811211776 2016-05-23 06:36:43 Pacific Daylight Time <Rob0tCunts> Alan watts?
734504329772695553 2016-05-22 15:00:53 Pacific Daylight Time <lookatyoubuddy> AYAYAYAYAYAYYAY YOOOOOOO LETS HACK THESE FUCKING SKIDSSS BRUOOHHOHOHO 😭
734503836883292160 2016-05-22 14:58:55 Pacific Daylight Time <lookatyoubuddy> @CVE20146271 SUP MY NIGGAAAAAAAAAA
734321883127156740 2016-05-22 02:55:54 Pacific Daylight Time <dongriat> @CVE20146271 Check your PM
731629995194523652 2016-05-14 16:39:18 Pacific Daylight Time <lookatyoubuddy> the party was just getting started :(
728983464113016832 2016-05-07 09:22:56 Pacific Dayli

c:\python37\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator SVC from version 0.22.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Tweet: Router vendor Linksys has locked user accounts on its Smart WiFi cloud service and is asking users to reset passwords after #hackers have been observed hijacking accounts and changing router settings to redirect users to #malware sites. #GMsectec   
Prediction: Relevant

Tweet: Shipping giant MSC discloses a malware-based attack       
Prediction: Relevant

Tweet: They say that VPNs, Antivirus, and Firewalls work hand and hand and u need all of them.. Firewalls protect your device's information (your files) but VPNs protect your personal information that u send via internet, and Antivirus kills viruses on your computer and malware
Prediction: Irrelevant

Tweet: RAT Malware Email Scam Threatens to Expose Your Online Activity     pic.twitter.com/zPmMTOnypi
Prediction: Relevant

Tweet: Google dice que los esc  neres de malware creados dentro del servicio de correo electr  nico gratuito Gmail bloquearon alrededor de 18 millones de correos electr  nicos de phishing y malware utilizan

In [2]:
#Get all security tags that we will be searching twitter with
def getTags():
    #Reads tags from file  and adds each tag to an array
    file = open("./security_tags.txt","r")
    arr = []
    line1 = file.readlines()
    for x in line1:
        arr.append(x)
    file.close()
    return arr

In [3]:
#Process the tags so they are ready to be used with Twint
def processTags(arr):
    #Removes the hashtag from every tag
    for i in range(len(arr)):
        arr[i] = arr[i][1:]
    file = open("./test1.json","w")
    file.close()
    return arr

In [4]:
#Use twint to collect tweets
def scrapeTweets(arr):
    #Iterates through all security terms and searches twitter, writes all the data to a file called test1.json
    for i in range(len(arr)):
        d = twint.Config()
        d.Search = arr[i]
        d.Limit = 20
        # d.Store_csv = True
        d.Store_object = True
        d.Store_json = True
        d.Output = "./test1.json"
        x = twint.run.Search(d)
    return 

In [5]:
#Process data collected from twint so it is ready to be encoded by bert
def processData():
    f= open("./test1.json","r",errors='ignore')
    line1 = f.readlines()
    arr = []
    for x in line1:
        arr.append(x)
    f.close() 
    arr.insert(0,"[")
    arr.append(']')
    for i in range(1,len(arr)-2):
        arr[i] = arr[i][:-1] + "," + arr[i][-1:]
    file = open("../test.json","w")
    for i in range(len(arr)):
        file.write(arr[i])
    file.close()
    f= open("../test.json","r",errors='ignore')
    line1 = f.readlines()
    data = []
    for x in line1:
        data.append(x)
    f.close()
    for i in range(len(data)):
        data[i] = (''.join([i if ord(i) < 128 else ' ' for i in str(data[i])]))
    file = open("./data.json","w")
    for i in range(len(data)):
        file.write(data[i])
    file.close()
    with open("./data.json", "r",errors='ignore') as read_file:
        data = json.load(read_file)
    return data

In [6]:
def getEncodings(data):
    #Remove lenght requirements as some of the tweets are longer and it will throw an error
    #Gets encodings from BaaS and creates a npy file with encodings called encoded_file.npy
    bc = BertClient(check_length = False)
    text_file_name = "new_text.npy"
    encoding_filename = "encoded_file.npy"
    np.save(text_file_name,data)
    print("Finished saving labels, beginning encoding process, this might take a couple minutes!\nWe will notify on completion")
    encodings = bc.encode(data)
    np.save(encoding_filename,encodings)
    print("Completed encoding and saving!")


In [7]:
#Prepare Data for encoding
def cleanData(data):
    #parse json data into two arrays containing the tweet texts and their correponding relevancy label
    #track numbers of relevant and irrelevant tweets 
    relnum =0
    irrnum = 0
    text = []
    labels = []
    newData = []
    seen_urls = set()
    duplicate_counter = 0
    
    for x in data:
#          Dont need to worry about duplicates for streaming
#         if re.sub(r"http\S+", "", x['tweet']) in seen_urls:
#             duplicate_counter +=1
#             continue
#         else:
#             seen_urls.add(re.sub(r"http\S+", "", x['tweet']))

        text.append(re.sub(r"http\S+", "", x['tweet']))
    

    for i in data:
        if(re.match('[a-zA-Z]',i['tweet'])):
            newData.append(i)
            continue
        else:
            continue
    newtext = [str for str in text if re.match('[a-zA-Z]', str)]
    data =newData
    getEncodings(newtext)
    return data

In [8]:
def classifyTweets(model_file,text_file,encoding_file,output_file,data):
    #Uses model stored in svm_model.p to get predictions and we add these predictions to our data array
    clf = pickle.load(open(model_file,'rb'))
    text = np.load(text_file)
    x = np.load(encoding_file)
    predictions = clf.predict(x)
    print_arr = ['Irrelevant','Relevant']
    f = open(output_file, 'w')
    f.close()
    counter = 0
    for i,t in enumerate(text):
        f = open(output_file, 'a')
        f.write("Tweet: "+t+'\n')
        f.write("Prediction: "+str(print_arr[predictions[i]])+'\n\n')
        f.close()
        data[counter]['Relevance'] = str(print_arr[predictions[i]])
        counter+=1
        print("Tweet: "+t)
        print("Prediction: "+str(print_arr[predictions[i]])+"\n")


In [9]:
def removeIrrelevant(data):
    #Cleans up our data by only keeping relevant tweets
    tempData = []
    for x in data:
        if x['Relevance'] == "Relevant":
            tempData.append(x)
        else:
            continue
    return tempData

In [10]:
def applyNLTK(data):
    #Runs all the tweets through nltk and stores all the nouns for each tweet in the data object
    for x in data:
        nouns = []
        tokens = nltk.word_tokenize(x['tweet'])
        tagged = nltk.pos_tag(tokens)
        for i in range(len(tagged)):
            if (tagged[i][1] == 'NNP'):
                nouns.append(tagged[i][0])
        x['nltk'] = nouns
    return data

In [11]:
def applySpacy(data):
    #Runs all the tweets through Spacy and stores the words extracted in the data object
    for x in data:
        doc = nlp(x['tweet'])
        x['spacy'] = [ent for ent in doc.ents]
    return data